In [ ]:
!kaggle datasets download sehriyarmemmedli/turboaz-cars-project

Dataset URL: https://www.kaggle.com/datasets/sehriyarmemmedli/turboaz-cars-project
License(s): unknown
100% 240M/240M [00:08<00:00, 37.4MB/s]
100% 240M/240M [00:08<00:00, 29.7MB/s]


In [ ]:
!unzip /content/turboaz-cars-project.zip

Archive:  /content/turboaz-cars-project.zip
  inflating: cars.csv                


Import

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures


from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

DataFrame

In [ ]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('/content/cars.csv')
df.head(6)

,id_x,car_rel_url_x,datetime_scrape,name,price_x,currency_x,datetime_product,city,day,hour,attributes,production_year,engine_displacement_num,engine_displacement_unit,kilometrage_num,kilometrage_unit,barter,loan,salon,spare_parts,vip,featured,img_url,id_y,cars_id,car_rel_url_y,datetime,description,price_y,currency_y,owner_name,shop_name,phone,updated,views,vin,car_details_id_x,Ban növü,Buraxılış ili,Hansı bazar üçün yığılıb,Marka,Model,Mühərrik,Qəzalı,Rəng,Sahiblər,Sürətlər qutusu,Vəziyyəti,Yeni,Yerlərin sayı,Yürüş,Ötürücü,Şəhər,car_details_id_y,car_rel_url,extra_info
0,3c234145-d57a-4ad6-9448-d43810fc3392,/autos/8748840-hyundai-i30,2024-09-13 20:32:19.751157+00,Hyundai i30,15000.0,AZN,"Bakı, dünən 23:28",bakı,13.09.2024,23:28,"2008, 1.6 L, 270 000 km",2008,1.6,L,270000,km,NaN,NaN,NaN,NaN,NaN,NaN,https://turbo.azstatic.com/uploads/f460x343/20...,8d84d800-fafd-4d5c-b640-f47bd6c5ac20,3c234145-d57a-4ad6-9448-d43810fc3392,/autos/8748840-hyundai-i30,2024-09-13 20:40:28.618345+00,Salam orjinal probeqdir bir ildi bizdedir biri...,15000.0,AZN,Şəmi,NaN,507687355.0,13.09.2024,492,NaN,8d84d800-fafd-4d5c-b640-f47bd6c5ac20,"Hetçbek, 5 qapı",2008,NaN,Hyundai,i30,1.6 L/115 a.g./Dizel,NaN,Gümüşü,2,Mexaniki,"Vuruğu yoxdur, rənglənməyib",Xeyr,5,270 000 km,Ön,Bakı,8d84d800-fafd-4d5c-b640-f47bd6c5ac20,/autos/8748840-hyundai-i30,Yüngül lehimli disklər* ABS* Mərkəzi qapanma* ...
1,c74ea36f-6be1-4de4-926d-e117197dcf00,/autos/8475807-lada-vaz-niva-travel,2024-09-13 20:32:19.751157+00,LADA (VAZ) Niva Travel,23700.0,AZN,"Bakı, dünən 23:59",bakı,13.09.2024,23:59,"2024, 1.7 L, 0 km",2024,1.7,L,0,km,NaN,NaN,Salon,NaN,vipped-icon,featured-icon,https://turbo.azstatic.com/uploads/f460x343/20...,2cf8b84b-adaf-467a-8f06-3dabcf866c8a,c74ea36f-6be1-4de4-926d-e117197dcf00,/autos/8475807-lada-vaz-niva-travel,2024-09-13 20:40:28.618345+00,LADA Niva Travel modelini nəğd və ya sərfəli l...,23700.0,AZN,NaN,Lada Azərbaycan,554092445.0,13.09.2024,60189,NaN,2cf8b84b-adaf-467a-8f06-3dabcf866c8a,"Offroader / SUV, 5 qapı",2024,Rəsmi diler,LADA (VAZ),Niva Travel,1.7 L/80 a.g./Benzin,NaN,Yaşıl,NaN,Mexaniki,"Vuruğu yoxdur, rənglənməyib",Bəli,5,0 km,Tam,Bakı,2cf8b84b-adaf-467a-8f06-3dabcf866c8a,/autos/8475807-lada-vaz-niva-travel,Yüngül lehimli disklər* ABS* Kondisioner* Otur...
2,9cefceb0-024d-4581-a869-a3c2c68a9f95,/autos/8739686-toyota-land-cruiser,2024-09-13 20:32:19.751157+00,Toyota Land Cruiser,35600.0,$,"Bakı, dünən 23:59",bakı,13.09.2024,23:59,"2011, 4.0 L, 164 750 km",2011,4.0,L,164750,km,NaN,NaN,NaN,NaN,vipped-icon,featured-icon,https://turbo.azstatic.com/uploads/f460x343/20...,22bb3704-ebe7-4357-ba2f-1806d1a7042b,9cefceb0-024d-4581-a869-a3c2c68a9f95,/autos/8739686-toyota-land-cruiser,2024-09-13 20:40:28.618345+00,Bezkraska.Mashinda hec bir problem yoxdur.Alve...,35600.0,USD,Anar,NaN,502126242.0,13.09.2024,2473,NaN,22bb3704-ebe7-4357-ba2f-1806d1a7042b,"Offroader / SUV, 5 qapı",2011,Rəsmi diler,Toyota,Land Cruiser,4.0 L/282 a.g./Benzin,NaN,Ağ,0,Avtomat,"Vuruğu yoxdur, rənglənməyib",Xeyr,8+,164 750 km,Tam,Bakı,22bb3704-ebe7-4357-ba2f-1806d1a7042b,/autos/8739686-toyota-land-cruiser,Yüngül lehimli disklər* ABS* Lyuk* Mərkəzi qap...
3,459cc337-fb63-48de-9694-41554923d311,/autos/8712597-hyundai-elantra,2024-09-13 20:32:19.751157+00,Hyundai Elantra,26700.0,AZN,"Bakı, dünən 23:59",bakı,13.09.2024,23:59,"2018, 2.0 L, 126 000 km",2018,2.0,L,126000,km,NaN,NaN,NaN,NaN,vipped-icon,NaN,https://turbo.azstatic.com/uploads/f460x343/20...,e0d16dac-4091-4417-916e-cadeff600f95,459cc337-fb63-48de-9694-41554923d311,/autos/8712597-hyundai-elantra,2024-09-13 20:40:28.618345+00,2019 alış\n,26700.0,AZN,Babək,NaN,774999004.0,13.09.2024,3727,5NPD84LFXKH406133,e0d16dac-4091-4417-916e-cadeff600f95,Sedan,2018,NaN,Hyundai,Elantra,2.0 L/150 a.g./Benzin,NaN,Boz,1,Avtomat,"Vuruğu yoxdur, rənglənməyib",Xeyr,NaN,126 000 km,Ön,Bakı,e0d16dac-4091-4417-916e-cadeff600f95,/autos/8712597-hyundai-elantra,Yüngül lehimli disklər* ABS* Lyuk* Yağış senso...
4,6c5ee8d8-1c6f-4fad-a694-957a4c43c25d,/autos/8674773-toyo

In [ ]:
df.columns

Index(['id_x', 'car_rel_url_x', 'datetime_scrape', 'name', 'price_x',
       'currency_x', 'datetime_product', 'city', 'day', 'hour', 'attributes',
       'production_year', 'engine_displacement_num',
       'engine_displacement_unit', 'kilometrage_num', 'kilometrage_unit',
       'barter', 'loan', 'salon', 'spare_parts', 'vip', 'featured', 'img_url',
       'id_y', 'cars_id', 'car_rel_url_y', 'datetime', 'description',
       'price_y', 'currency_y', 'owner_name', 'shop_name', 'phone', 'updated',
       'views', 'vin', 'car_details_id_x', 'Ban növü', 'Buraxılış ili',
       'Hansı bazar üçün yığılıb', 'Marka', 'Model', 'Mühərrik', 'Qəzalı',
       'Rəng', 'Sahiblər', 'Sürətlər qutusu', 'Vəziyyəti', 'Yeni',
       'Yerlərin sayı', 'Yürüş', 'Ötürücü', 'Şəhər', 'car_details_id_y',
       'car_rel_url', 'extra_info'],
      dtype='object')

In [ ]:
# price_x, price_y: qiymət
# currency_x, currency_y: valyuta
# attributes: avtomobil haqqında məlumatlar (istehsal ili, mühərrik həcmi, getdiyi kilometr)
# city, Şəhər: şəhər
# day: hansı gündə elan qoyulub (yenilənib)
# hour: hansı saatda elan qoyulub (yenilənib)
# vip: elanın tipi (vip elan vs. normal elan)
# featured: elanın tipi (önə çəkilmiş elan vs. normal elan)
# barter: barterin mümkün olub olmaması
# loan: avtomobilin kreditdə olub olmaması
# salon: elanı verənin salon olub olmaması
# description: avtomobil haqqında açıqlama
# owner_name: sahibin adı
# shop_name: mağazanın adı
# phone: telefon nömrəsi
# vin: avtomobilin identifikasiya nömrəsi
# Ban növü
# Buraxılış ili
# Hansı bazar üçün yığılıb
# Marka
# Model
# Mühərrik
# Qəzalı, spare_parts: qəzalı olub olmaması
# Rəng
# Sahiblər
# Sürətlər qutusu Vəziyyəti
# Yeni
# Yerlərin sayı
# Yürüş
# Ötürücü
# extra_info: əlavə məlumatlar

In [ ]:
# 'id_x',     --------- sil
# 'car_rel_url_x', ---------- sil
# 'datetime_scrape', ---------- sil
# 'name', ------------ sil
# 'price_x',  +
# 'currency_x',+
# 'datetime_product', ------------- sil
# 'city', ------- evez et
# 'day', --------- sil
# 'hour', ---------- sil
# 'attributes', -------- sil
# 'production_year', +
# 'engine_displacement_num', +
# 'engine_displacement_unit', ------- sil
# 'kilometrage_num', +
# 'kilometrage_unit', ------- sil
# 'barter', +
# 'loan', +
# 'salon', +
# 'spare_parts', +
# 'vip', +
# 'featured', +
# 'img_url', ----------- sil
# 'id_y', --------- sil
# 'cars_id', -------- sil
# 'car_rel_url_y', ----------sil
# 'datetime', -------- sil
# 'description',
# 'price_y', ----------- sil
# 'currency_y', -------------sil
# 'owner_name', ------------- sil
# 'shop_name', ----------- sil
# 'phone', ---------- sil
# 'updated', ---------- sil
# 'views', +
# 'vin', ----------- sil
# 'car_details_id_x', -------- sil
# 'Ban növü', ------ hisselere bol
# 'Buraxılış ili', ----------- sil
# 'Hansı bazar üçün yığılıb', +
# 'Marka', +
# 'Model', +
# 'Mühərrik',  ------------ a.g ayir
# 'Qəzalı',------------- sil
# 'Rəng',
# 'Sahiblər', +
# 'Sürətlər qutusu', +
# 'Vəziyyəti', +
# 'Yeni', +
# 'Yerlərin sayı', +
# 'Yürüş',  ------------- sil
# 'Ötürücü', +
# 'Şəhər', ----------- sil
# 'car_details_id_y', --------- sil
# 'car_rel_url', -------- sil
# 'extra_info' -------- bol

# Duplicates

In [ ]:
df['cars_id'].value_counts()
# burda tekrarlanan setirler gorunur

,count
cars_id,
4be1e7e3-1222-4b55-89e2-acf8934d7ff8,2
9dc9d6e3-3f2d-47c3-8eb0-49d0cf2455cb,2
0d30f033-2217-4b91-a63b-1fd1aacdcd69,2
1aba14c8-482d-48b7-96c6-2856678249ff,2
e60b412e-48fc-47bf-8ba8-c286cfcdad5f,2
...,...
b024991f-7711-47da-a8c8-bc61a4b79703,1
f483b88c-11df-4f73-96b3-2c8c3f243569,1
6b26b7f7-e205-469d-8ae3-682e8d32999e,1


In [ ]:
df[df['cars_id']=='0d30f033-2217-4b91-a63b-1fd1aacdcd69'][['price_x', 'Marka', 'Model', 'description']]

,price_x,Marka,Model,description
149025,49000.0,Jeep,Wrangler,"{""Tok rejimi ile 40-50 km gede bilir. Hec bir ..."
149026,49000.0,Jeep,Wrangler,"{""Tok rejimi ile 40-50 km gede bilir. Hec bir ..."


In [ ]:
df.drop_duplicates(subset=['city', 'production_year', 'engine_displacement_num',
       'kilometrage_num', 'barter', 'loan', 'salon',  'featured', 'shop_name', 'Ban növü',
       'Hansı bazar üçün yığılıb', 'Marka', 'Model', 'Mühərrik',
       'Rəng', 'Sahiblər', 'Sürətlər qutusu', 'Vəziyyəti', 'Yeni',
       'Yerlərin sayı', 'Ötürücü'], keep='last', inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df['currency_x'].unique()

array(['AZN', '$', '€'], dtype=object)

In [ ]:
def currency(x):
  if x['currency_x'] == 'AZN':
    return x['price_x']
  if x['currency_x'] == '$':
    return x['price_x']*1.7
  if x['currency_x'] == '€':
    return x['price_x']*1.85

df['price'] = df.apply(currency, axis=1)        # valyutani deyisib hamisini AZN-e cevirir

In [ ]:
df.drop(['id_x', 'car_rel_url_x', 'datetime_scrape', 'name', 'price_x', 'currency_x', 'datetime_product', 'day', 'hour', 'attributes',
        'engine_displacement_unit', 'kilometrage_unit', 'img_url', 'id_y', 'cars_id', 'car_rel_url_y', 'datetime', 'price_y', 'currency_y',
         'owner_name', 'shop_name', 'phone', 'updated', 'vin', 'car_details_id_x', 'Buraxılış ili', 'Qəzalı', 'Yürüş', 'Şəhər',
         'car_details_id_y', 'car_rel_url'], axis=1, inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.shape

(198624, 26)

In [ ]:
df = df.sample(n=50000, random_state=7).reset_index(drop=True)
df.shape

(50000, 26)

# Preprocessing

In [ ]:
X = df.drop('price', axis=1)
y = df['price'].copy()            #targeti ayiririq

In [ ]:
X['extra_info'] = X['extra_info'].apply(lambda x: [i.strip() for i in x.split('*')])   # * isaresine gore ayiririq
unique_features = set([info for sublist in X['extra_info'] for info in sublist])
for info in unique_features:
    X[info] = X['extra_info'].apply(lambda x: info in x)
    X[info] = X[info].astype(int)

X = X.drop('extra_info', axis=1)  #ehtiyac olmadigina gore silirik
# her xususiyyeti ayri sutun kimi gosterib, varsa 1, yoxdusa 0 kimi isare edirik

In [ ]:
X.shape

(50000, 37)

In [ ]:
X[['type', 'door']] = X['Ban növü'].str.split(',', expand=True)
X['type'] = X['type'].str.strip()
X['door'] = X['door'].str.strip()
# ban novunu bolduk, iki sutunda gosterdik
X.drop('Ban növü', axis=1, inplace=True)   #artiq ehtiyac olmadigi ucun sildik

In [ ]:
X['door'].unique()

array([None, '3 qapı', '5 qapı', 'ikiqat kabin', 'açıq',
       'bir yarım kabin', '4 qapı', 'tək kabin'], dtype=object)

In [ ]:
def process_value(value):
    if value is not None and isinstance(value, str):
        match = re.search(r'\d+', value)
        return int(match.group()) if match else np.nan
    else:
        return np.nan

X['door'] = X['door'].apply(process_value)
# nece qapili oldugunu cixardiq, lazimsiz deyerleri sildik

In [ ]:
X['door'].unique()

array([nan,  3.,  5.,  4.])

In [ ]:
X['door'].dtype

dtype('float64')

In [ ]:
X['door'].value_counts()

,count
door,
5.0,18660
3.0,659
4.0,38


In [ ]:
X[['muherrik_hecmi', 'at_gucu', 'yanacaq_novu']] = X['Mühərrik'].str.split('/', expand=True)
X['at_gucu'] = X['at_gucu'].str.strip()
X['yanacaq_novu'] = X['yanacaq_novu'].str.strip()
X.drop(['muherrik_hecmi', 'Mühərrik'], axis=1, inplace=True)
# muherrik hecmi tekrar sutun olacagi ucun silirem

In [ ]:
X['at_gucu'] = X['at_gucu'].str.replace(' a.g.', '', regex = True)
X['at_gucu'] = pd.to_numeric(X['at_gucu'], errors='coerce')
# sutundan herfleri silib tipini deyisirik

In [ ]:
X['at_gucu'].dtype

dtype('float64')

In [ ]:
df['city'].value_counts().head(10)  #esas hisse bakidir

,count
city,
bakı,36916
sumqayıt,3460
gəncə,1879
xırdalan,469
lənkəran,404
şəmkir,357
xaçmaz,351
bərdə,317
mingəçevir,315


In [ ]:
X['city'] = X['city'].apply(lambda x:1 if x == 'bakı' else 0)

In [ ]:
X['city'].dtype

dtype('int64')

In [ ]:
list(X['description'])[:30]   # sutunda melumatlardan keywordler cixarmaq ucun melumatlarla tanis oluruq

['Hunday sonata sport Limited versiya\n-Keyless Go(açarsız daxil olma), Start Stop\n-Bütöv panaramik Tavan\n- Qara zavod dam örtüyü\n- Böyük manitor(touch screen)\n- Apple carplay,android carplay,Usb flaska,aux,bluetooth\n- Multi Sport rul( +- lipstoklar)\n- Autohold\n- Elektron əl əyləci\n- Park radar\n- Distronic avtopilot( qarşıdakı maşınla ara məsafəsi)\n- Zolaq tanıma(yol xətti izləmə)\n- Qəza önləyici sistem \n- Enişə və yoxuşa dəstək sistemi\n- Arxa park radar\n- İşığa həssas qaralan güzgülər\n- Avtomatik uzaq işıqların yaxına keçməsi\n- Güzgülərdə Ölü zona(kor zona)\n- Ön və arxa oturacaqların isidilməsi ön oturacaqların soyudulması və elektron tənzimlənməsi\n- Ön oturacağın yaddaş paketi\n- İnfinity səs sistemi (xüsusi səs sistemidi)\n- Eco,Smart,Sport rejimlər\n- Yağış və Işıq sensoru\n- Zavod arxa qapıdan çıxan pərdələr\n    Maşın ən full buraxılışdır orginal 163000 km yürüşü var Amerkadan gələnnən yağın Luqi Molynən dəyişmişəm bütün lazımı qulluqlar olunubdur Hava filtiri s

In [ ]:
# # elemek istedim ram icaze vermedi
# X['description'] = X['description'].fillna('')
# vectorizer = TfidfVectorizer()
# vectorized = vectorizer.fit_transform(X['description'])
# dense_matrix = vectorized.todense()
# tfidf_df = pd.DataFrame(dense_matrix, columns=vectorizer.get_feature_names_out())
# X = pd.concat([X, tfidf_df], axis=1)
# X.head(3)

In [ ]:
# positive keywordleri ayiririq
positive_words = ['ideal', 'normal', 'orginal', 'yeni', 'full', 'təzə', 'əla', 'ela', 'problemi yoxdu']

# metnin icinden positive keywordleri ayirmaq ucun funksiya yaziriq
def extract_positive_words(text, positive_words):
    if not isinstance(text, str):     #xanadaki melumat string deyilse '' qaytarir
        return ''

    # metnin icinde positive keywordlerin olub-olmamasini yoxlayiriq
    found_words = [word for word in positive_words if re.search(r'\b' + re.escape(word) + r'\b', text, re.IGNORECASE)]
    return ' '.join(found_words)

# funksiyani sutuna tetbiq edirik
X['positive_words'] = X['description'].apply(lambda x: extract_positive_words(x, positive_words))

,positive_words_count,positive_words
0,2,"[orginal, full]"
1,0,[]
2,0,[]
3,2,"[yeni, full]"
4,1,[təzə]
...,...,...
49995,0,[]
49996,0,[]
49997,1,[ideal]
49998,0,[]


In [ ]:
# Eynile positive keywordleri tapdigimiz kimi negativleri de tapiriq
negative_words = ['kosmetik', 'kasmetik', 'cızıq', 'təcili', 'problemi var', 'xərc']

def extract_negative_words(text, positive_words):
    if not isinstance(text, str):
        return ''
    found_words = [word for word in negative_words if re.search(r'\b' + re.escape(word) + r'\b', text, re.IGNORECASE)]
    return ' '.join(found_words)


X['negative_words'] = X['description'].apply(lambda x: extract_negative_words(x, negative_words))

In [ ]:
X['negative_words'] = X['negative_words'].str.split(' ')
X['negative_words_count'] = X['negative_words'].apply(lambda x: len([i for i in x if i]) if isinstance(x, list) else 0)
X[['negative_words_count', 'negative_words']]

,negative_words_count,negative_words
0,0,[]
1,0,[]
2,0,[]
3,0,[]
4,0,[]
...,...,...
49995,0,[]
49996,0,[]
49997,0,[]
49998,0,[]


In [ ]:
# musbet xususiyyetlerle menfilerin ferqini tapiriq
X['pos_neg_words_count'] = X['positive_words_count'] - X['negative_words_count']

In [ ]:
# artiq lazim olmayacaq sutunlari silirik
X.drop(['positive_words', 'description', 'positive_words_count', 'negative_words', 'negative_words_count'], axis=1, inplace=True)

In [ ]:
X['pos_neg_words_count'].dtype

dtype('int64')

In [ ]:
X.shape

(50000, 39)

In [ ]:
X.dtypes

,0
city,int64
production_year,int64
engine_displacement_num,float64
kilometrage_num,int64
barter,object
loan,object
salon,object
spare_parts,object
vip,object
featured,object


# Model

In [ ]:
# Datani train ve test hisselerine boluruk
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=7)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=7)

In [ ]:
X_train.shape, X_valid.shape, X_test.shape

((40000, 39), (5000, 39), (5000, 39))

In [ ]:
# numeric ve categoric sutunlarive sutun adlarini ayiririq
X_train_num = X_train.select_dtypes(include=[np.number])
X_train_cat = X_train.select_dtypes(exclude=[np.number])
num_features = X_train_num.columns
cat_features = X_train_cat.columns

In [ ]:
num_pipeline = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler(),
)

cat_pipeline = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
)

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features),
])

X_train_prepared = full_pipeline.fit_transform(X_train)
X_test_prepared = full_pipeline.transform(X_test)
X_valid_prepared = full_pipeline.transform(X_valid)

In [ ]:
X_train_prepared.shape, X_test_prepared.shape, X_valid_prepared.shape


((40000, 1901), (5000, 1901), (5000, 1901))

In [ ]:
import tensorflow as tf

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = tf.keras.Sequential(
    [tf.keras.layers.Input(shape=(1901,)),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(150, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1)]
)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3)
model.compile(loss='mean_absolute_error',
              optimizer=optimizer,
              metrics=['RootMeanSquaredError', 'MeanSquaredError'])
history = model.fit(X_train_prepared, y_train, epochs=25,
                    validation_data=(X_valid_prepared, y_valid))

Epoch 1/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 16ms/step - MeanSquaredError: 46819588.0000 - RootMeanSquaredError: 6752.2046 - loss: 2282.9600 - val_MeanSquaredError: 90798800.0000 - val_RootMeanSquaredError: 9528.8408 - val_loss: 3770.6335
Epoch 2/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - MeanSquaredError: 45472116.0000 - RootMeanSquaredError: 6648.7100 - loss: 2258.5000 - val_MeanSquaredError: 93624264.0000 - val_RootMeanSquaredError: 9675.9629 - val_loss: 3862.5696
Epoch 3/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - MeanSquaredError: 45470232.0000 - RootMeanSquaredError: 6655.9204 - loss: 2270.7021 - val_MeanSquaredError: 93908504.0000 - val_RootMeanSquaredError: 9690.6396 - val_loss: 3843.3635
Epoch 4/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - MeanSquaredError: 45565200.0000 - RootMeanSquaredError: 6666.3623 - loss: 2228.5151 - val_MeanSquaredError: 96407640.0000 - val_RootMeanSquaredError: 9818.7393 - val_loss: 3921.1177
Epoch 5/25
1250/1250 ━━━━━━━━━━━━━━━━━━━